In [1]:
import os
from dotenv import load_dotenv
import openai

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
from llama_index.core import SimpleDirectoryReader

In [9]:
documents = SimpleDirectoryReader(
    input_files = ["./docs/PLAID.pdf"]
).load_data()

In [14]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

10 

<class 'llama_index.core.schema.Document'>
Doc ID: d6151871-a842-4fa6-bb5c-c0c9cfd5b5b2
Text: PLAID: An Efficient Engine for Late Interaction Retrieval Keshav
Santhanam∗ keshav2@stanford.edu Stanford University United StatesOmar
Khattab∗ okhattab@stanford.edu Stanford University United States
Christopher Potts Stanford University United StatesMatei Zaharia
Stanford University United States ABSTRACT Pre-trained language models
are increas...


In [15]:
# print all documents length with a delay i.e. clear the screen after printing each document
import time
for doc in documents:
    print(doc)
    time.sleep(1)
    os.system('clear')

Doc ID: d6151871-a842-4fa6-bb5c-c0c9cfd5b5b2
Text: PLAID: An Efficient Engine for Late Interaction Retrieval Keshav
Santhanam∗ keshav2@stanford.edu Stanford University United StatesOmar
Khattab∗ okhattab@stanford.edu Stanford University United States
Christopher Potts Stanford University United StatesMatei Zaharia
Stanford University United States ABSTRACT Pre-trained language models
are increas...
Doc ID: eec9fb0e-e4a5-4923-8de7-b1f671c546f4
Text: PLAID, May 2022, Preprint Keshav Santhanam*, Omar Khattab*,
Christopher Potts, and Matei Zaharia Question Passage Question Encoder
Passage EncoderMaxSim MaxSim MaxSimscore Offline Indexing Figure 1:
The late interaction architecture, given a query and a passage.
Diagram from Khattab et al. [21] with permission. interaction with the
residual repr...
Doc ID: 531d8b11-edee-4f1b-8ae1-97fd2cc9ec67
Text: PLAID: An Efficient Engine for Late Interaction Retrieval PLAID,
May 2022, Preprint MS MARCO v1 LoTTE pooled 20 40 60 80100 120 140 #
Passages0.6

In [18]:
len(documents)

10

In [20]:
from llama_index.core import Document

In [21]:
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [22]:
print(document)

Doc ID: b7e1bf98-d0ce-41ec-bd81-9d1ba792b16b
Text: PLAID: An Efficient Engine for Late Interaction Retrieval Keshav
Santhanam∗ keshav2@stanford.edu Stanford University United StatesOmar
Khattab∗ okhattab@stanford.edu Stanford University United States
Christopher Potts Stanford University United StatesMatei Zaharia
Stanford University United States ABSTRACT Pre-trained language models
are increas...


In [27]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

/var/folders/0j/d0xd373d2tb7zdzcsqrk3mpw0000gn/T/ipykernel_24193/339795532.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(
/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
index = VectorStoreIndex.from_documents(
    [document],
    service_context=service_context
    )

In [30]:
query_engine = index.as_query_engine()

In [31]:
response = query_engine.query(
    "what are stages in PLAID pipeline?"
)
print(str(response))

The stages in the PLAID pipeline are as follows:
1. Centroid interaction without pruning
2. Centroid interaction with centroid pruning
3. Optimized kernels for efficient deployment of centroid interaction and MaxSim operations
